In [28]:
from functions import get_sku_list
from facebook_business.adobjects.productcatalog import ProductCatalog
from facebook_business.adobjects.productgroup import ProductGroup
from facebook_business.adobjects.productitem import ProductItem
from facebook_business.api import FacebookAdsApi


def check_product_availability(page: str, number_of_pages: int, product_catalog_id: int, access_token: str) -> dict:
    sku_list = get_sku_list(page, num_pages=number_of_pages)
    FacebookAdsApi.init(access_token=access_token)
    oscar_catalog = ProductCatalog(product_catalog_id)
    products = oscar_catalog.get_products(fields=[ProductItem.Field.product_group, ProductItem.Field.name, ProductItem.Field.availability], params={
                                          'filter': {'retailer_id': {'is_any': sku_list}}})
    new_products = [[dict(p) for p in ProductGroup(product['product_group']['id']).get_products(
        fields=[ProductItem.Field.name, ProductItem.Field.availability])] for product in products]

    list_brokens = [k['is_broken'] for k in products]

    i = 0
    for k in list_brokens:
        if k == 'NO':
            i = i + 1

    print("Produtos não quebrados:", i)
    print("Produtos quebrados:", (len(products) - i))
    print(('Porcentagem de produtos quebrados em relação ao total:', len(products)-i)/len(products)*100, )
    print("Produtos", len(products))

    return [{'product_name': group[0]['name'],
            'product_quantity': len(group),
             'products_available': len([p for p in group if p['availability'] != 'out of stock']),
             'is_broken': 'YES' if len([p for p in group if p['availability'] == 'out of stock']) >= len(group) / 2 else 'NO'
             } for group in new_products
            ]


products = check_product_availability(page='', number_pages=1)


7 Produtos não quebrados
14 Produtos quebrados
21 Produtos
